In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from tqdm import tqdm

import requests
from bs4 import BeautifulSoup
from io import StringIO

from data.omni import OMNI
from data.wsa_enlil import WSA_ENLIL
from data.eswf3_2 import ESWF3_2
from data.dl_model import DL_Model

from scipy.stats import pearsonr

#CR_file_path = 'E:/Research/SWspeed/Carrington_Rotation_Data__Oct-Dec__2010-2020_.csv'
#CR_df = pd.read_csv(CR_file_path)

## ICME Events

In [7]:
# ICME event list URL
ICME_url = "https://izw1.caltech.edu/ACE/ASC/DATA/level3/icmetable2.htm"

# URL에서 HTML 페이지 가져오기
response = requests.get(ICME_url)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table')
    icme_entire_table = pd.read_html(StringIO(str(table)))[0]
    
    # 열 선택 (ICME_start, ICME_end, ICME_mean, ICME_max)
    icme_sub_table = icme_entire_table.iloc[:, [1, 2, 11, 12]]
    new_column_names = ['ICME_start', 'ICME_end', 'ICME_mean', 'ICME_max']  # column 이름 설정
    icme_sub_table.columns = new_column_names
    icme_sub_table = icme_sub_table.dropna(subset=['ICME_start', 'ICME_end'])  # NaN 제거

    # datetime format
    icme_sub_table['ICME_start'] = pd.to_datetime(icme_sub_table['ICME_start'], errors='coerce', format='%Y/%m/%d %H%M')
    icme_sub_table['ICME_end'] = pd.to_datetime(icme_sub_table['ICME_end'], errors='coerce', format='%Y/%m/%d %H%M')

    # filtering (Oct-Dec, 2012, 2020)
    filtered_icme_df = icme_sub_table[
        (icme_sub_table['ICME_start'].dt.year >= 2012) & 
        (icme_sub_table['ICME_start'].dt.year <= 2020) & 
        (icme_sub_table['ICME_start'].dt.month >= 10) & 
        (icme_sub_table['ICME_start'].dt.month <= 12) & 
        (icme_sub_table['ICME_end'].dt.year >= 2012) & 
        (icme_sub_table['ICME_end'].dt.year <= 2020) & 
        (icme_sub_table['ICME_end'].dt.month >= 10) & 
        (icme_sub_table['ICME_end'].dt.month <= 12)
    ]
    filtered_icme_df.reset_index(drop=True, inplace=True)
    icme_df = filtered_icme_df

else:
    print(f"Request failed with status code: {response.status_code}")
    
icme_event_list = []

for i in range(len(icme_df['ICME_start'])):
    ICME_event = [
        icme_df['ICME_start'][i].strftime('%Y-%m-%d %H:%M'), 
        icme_df['ICME_end'][i].strftime('%Y-%m-%d %H:%M')
    ]
    icme_event_list.append(ICME_event)

#icme_event_list=icme_event_list[2:] # DL model에 없는 날짜 제거
print("Number of the ICME events:", len(icme_event_list))
icme_event_list

Number of the ICME events: 32


[['2012-10-01 00:00', '2012-10-02 00:00'],
 ['2012-10-02 22:00', '2012-10-03 06:00'],
 ['2012-10-08 18:00', '2012-10-09 12:00'],
 ['2012-10-12 22:00', '2012-10-13 10:00'],
 ['2012-11-01 00:00', '2012-11-02 03:00'],
 ['2012-11-09 03:00', '2012-11-09 15:00'],
 ['2012-11-13 08:00', '2012-11-14 03:00'],
 ['2012-11-24 12:00', '2012-11-25 10:00'],
 ['2012-11-26 12:00', '2012-11-28 05:00'],
 ['2012-11-28 18:00', '2012-11-29 14:00'],
 ['2012-12-14 07:00', '2012-12-14 17:00'],
 ['2013-10-02 23:00', '2013-10-03 22:00'],
 ['2013-10-09 09:00', '2013-10-11 00:00'],
 ['2013-11-08 22:00', '2013-11-09 07:00'],
 ['2013-11-11 17:00', '2013-11-12 03:00'],
 ['2013-12-01 11:00', '2013-12-02 23:00'],
 ['2013-12-15 16:00', '2013-12-16 05:00'],
 ['2013-12-25 05:00', '2013-12-25 17:00'],
 ['2014-11-11 07:00', '2014-11-11 20:00'],
 ['2014-12-22 04:00', '2014-12-22 17:00'],
 ['2015-10-25 14:00', '2015-10-27 04:00'],
 ['2015-11-04 15:00', '2015-11-04 19:00'],
 ['2015-11-07 06:00', '2015-11-08 16:00'],
 ['2015-12-

---

## Statistical Verification

### MAE

In [14]:
all_CR_list = [2128, 2129, 2130, 2131, 2132,
               2142, 2143, 2144, 2145, 2155, 2156, 2157, 2158, 
               2168, 2169, 2170, 2171, 2172, 2182, 2183, 2184, 2185, 2195, 2196, 2197, 2198, 2199, 2209, 2210, 2211, 2212, 
               2222, 2223, 2224, 2225, 2235, 2236, 2237, 2238, 2239]

ascending_CR_list = [2128, 2129, 2130, 2131, 2132]
maximum_CR_list = [2142, 2143, 2144, 2145, 2155, 2156, 2157, 2158]
descending_CR_list = [2168, 2169, 2170, 2171, 2172, 2182, 2183, 2184, 2185, 2195, 2196, 2197, 2198, 2199, 2209, 2210, 2211, 2212]
minimum_CR_list = [2222, 2223, 2224, 2225, 2235, 2236, 2237, 2238, 2239]

model_mae = 0
wsa_mae = 0
eswf_mae = 0

model_n = 0
wsa_n = 0
eswf_n = 0

def MAE(omni, model, valid_index):
    omni = np.array(omni)
    model = np.array(model)
    
    errors = omni[valid_index] - model[valid_index]
    error_sum = np.sum(np.abs(errors))
    n = len(errors)
    return error_sum, n

for CR_lists in [all_CR_list, ascending_CR_list, maximum_CR_list, descending_CR_list, minimum_CR_list]:
    for CR in tqdm(CR_lists):
        _, omni_speeds = OMNI(CR)
        _, wsa_speeds = WSA_ENLIL(CR)
        _, eswf3_2speeds = ESWF3_2(CR)
        _, model_speeds = DL_Model(CR)
    
        valid_index = ~np.isnan(omni_speeds) & ~np.isnan(model_speeds) & ~np.isnan(wsa_speeds) & ~np.isnan(eswf3_2speeds)
    
        # 사용자 모델에 대한 MAE 계산
        model_error_sum, model_count = MAE(omni_speeds, model_speeds, valid_index)
        model_mae += model_error_sum
        model_n += model_count
        
        # WSA 모델에 대한 MAE 계산
        wsa_error_sum, wsa_count = MAE(omni_speeds, wsa_speeds, valid_index)
        wsa_mae += wsa_error_sum
        wsa_n += wsa_count
    
        # ESWF3_2 모델에 대한 MAE 계산
        eswf_error_sum, eswf_count = MAE(omni_speeds, eswf3_2speeds, valid_index)
        eswf_mae += eswf_error_sum
        eswf_n += eswf_count

        #print(np.round(model_error_sum/model_count, 4))

    print("model: {} km/s".format(np.round(model_mae/model_n, 4)))
    print("wsa: {} km/s".format(np.round(wsa_mae/wsa_n, 1)))
    print("eswf: {} km/s".format(np.round(eswf_mae/eswf_n, 1)), '\n')

    model_mae, wsa_mae, eswf_mae = 0, 0, 0
    model_n, wsa_n, eswf_n = 0, 0, 0

  0%|                                                                                            | 0/5 [01:54<?, ?it/s]


KeyboardInterrupt: 

### RMSE

In [ ]:
all_CR_list = [2128, 2129, 2130, 2131, 2132,
               2142, 2143, 2144, 2145, 2155, 2156, 2157, 2158, 
               2168, 2169, 2170, 2171, 2172, 2182, 2183, 2184, 2185, 2195, 2196, 2197, 2198, 2199, 2209, 2210, 2211, 2212, 
               2222, 2223, 2224, 2225, 2235, 2236, 2237, 2238, 2239]

ascending_CR_list = [2128, 2129, 2130, 2131, 2132]
maximum_CR_list = [2142, 2143, 2144, 2145, 2155, 2156, 2157, 2158]
descending_CR_list = [2168, 2169, 2170, 2171, 2172, 2182, 2183, 2184, 2185, 2195, 2196, 2197, 2198, 2199, 2209, 2210, 2211, 2212]
minimum_CR_list = [2222, 2223, 2224, 2225, 2235, 2236, 2237, 2238, 2239]

model_mse = 0
wsa_mse = 0
eswf_mse = 0

model_n = 0
wsa_n = 0
eswf_n = 0

def MSE(omni, model, valid_index):
    omni = np.array(omni)
    model = np.array(model)
    
    errors = omni[valid_index] - model[valid_index]
    error_sum = np.sum(errors ** 2)
    n = len(errors)
    return error_sum, n

for CR_lists in [all_CR_list, ascending_CR_list, maximum_CR_list, descending_CR_list, minimum_CR_list]:
    for CR in tqdm(CR_lists):
        _, omni_speeds = OMNI(CR)
        _, wsa_speeds = WSA_ENLIL(CR)
        _, eswf3_2speeds = ESWF3_2(CR)
        _, model_speeds = DL_Model(CR)
        
        valid_index = ~np.isnan(omni_speeds) & ~np.isnan(model_speeds) & ~np.isnan(wsa_speeds) & ~np.isnan(eswf3_2speeds)
        
        # 사용자 모델에 대한 MSE 계산
        model_error_sum, model_count = MSE(omni_speeds, model_speeds, valid_index)
        model_mse += model_error_sum
        model_n += model_count
        
        # WSA 모델에 대한 MSE 계산
        wsa_error_sum, wsa_count = MSE(omni_speeds, wsa_speeds, valid_index)
        wsa_mse += wsa_error_sum
        wsa_n += wsa_count
    
        # ESWF3_2 모델에 대한 MSE 계산
        eswf_error_sum, eswf_count = MSE(omni_speeds, eswf3_2speeds, valid_index)
        eswf_mse += eswf_error_sum
        eswf_n += eswf_count

        #print(np.round(np.sqrt(model_error_sum/model_count), 4))
    
    # RMSE 계산
    print("model: {} km/s".format(np.round(np.sqrt(model_mse/model_n), 4)))
    print("wsa: {} km/s".format(np.round(np.sqrt(wsa_mse/wsa_n), 1)))
    print("eswf: {} km/s".format(np.round(np.sqrt(eswf_mse/eswf_n), 1)), '\n')

    model_mse, wsa_mse, eswf_mse = 0, 0, 0
    model_n, wsa_n, eswf_n = 0, 0, 0

### PCC

In [ ]:
all_CR_list = [2128, 2129, 2130, 2131, 2132,
               2142, 2143, 2144, 2145, 2155, 2156, 2157, 2158, 
               2168, 2169, 2170, 2171, 2172, 2182, 2183, 2184, 2185, 2195, 2196, 2197, 2198, 2199, 2209, 2210, 2211, 2212, 
               2222, 2223, 2224, 2225, 2235, 2236, 2237, 2238, 2239]

ascending_CR_list = [2128, 2129, 2130, 2131, 2132]
maximum_CR_list = [2142, 2143, 2144, 2145, 2155, 2156, 2157, 2158]
descending_CR_list = [2168, 2169, 2170, 2171, 2172, 2182, 2183, 2184, 2185, 2195, 2196, 2197, 2198, 2199, 2209, 2210, 2211, 2212]
minimum_CR_list = [2222, 2223, 2224, 2225, 2235, 2236, 2237, 2238, 2239]

wsa_omni_valid = []
wsa_model_valid = []

eswf_omni_valid = []
eswf_model_valid = []

model_omni_valid = []
model_model_valid = []

for CR_lists in [all_CR_list, ascending_CR_list, maximum_CR_list, descending_CR_list, minimum_CR_list]:
    for CR in tqdm(CR_lists):
        _, omni_speeds = OMNI(CR)
        _, wsa_speeds = WSA_ENLIL(CR)
        _, eswf3_2speeds = ESWF3_2(CR)
        _, model_speeds = DL_Model(CR)
        
        omni_speeds = np.array(omni_speeds)
        wsa_speeds = np.array(wsa_speeds)
        eswf3_2speeds = np.array(eswf3_2speeds)
        model_speeds = np.array(model_speeds)
    
        valid_index = ~np.isnan(omni_speeds) & ~np.isnan(model_speeds) & ~np.isnan(wsa_speeds) & ~np.isnan(eswf3_2speeds)
        
        # 사용자 모델에 대한 유효한 데이터 수집
        model_omni_valid.extend(omni_speeds[valid_index])
        model_model_valid.extend(model_speeds[valid_index])
        
        # WSA 모델에 대한 유효한 데이터 수집
        wsa_omni_valid.extend(omni_speeds[valid_index])
        wsa_model_valid.extend(wsa_speeds[valid_index])
        
        # ESWF3_2 모델에 대한 유효한 데이터 수집
        eswf_omni_valid.extend(omni_speeds[valid_index])
        eswf_model_valid.extend(eswf3_2speeds[valid_index])
        
    model_pcc, _ = pearsonr(model_omni_valid, model_model_valid)
    wsa_pcc, _ = pearsonr(wsa_omni_valid, wsa_model_valid)
    eswf_pcc, _ = pearsonr(eswf_omni_valid, eswf_model_valid)
    
    # PCC 계산
    print("model PCC: {}".format(np.round(model_pcc, 4)))
    print("wsa PCC: {}".format(np.round(wsa_pcc, 2)))
    print("eswf PCC: {}".format(np.round(eswf_pcc, 2)), '\n')
    
    wsa_omni_valid = []
    wsa_model_valid = []

    eswf_omni_valid = []
    eswf_model_valid = []

    model_omni_valid = []
    model_model_valid = []

In [ ]:
CR_list_2012 = [2128, 2129, 2130, 2131, 2132]
CR_list_2013 = [2142, 2143, 2144, 2145]
CR_list_2014 = [2155, 2156, 2157, 2158]
CR_list_2015 = [2168, 2169, 2170, 2171, 2172]
CR_list_2016 = [2182, 2183, 2184, 2185]
CR_list_2017 = [2195, 2196, 2197, 2198, 2199]
CR_list_2018 = [2209, 2210, 2211, 2212]
CR_list_2019 = [2222, 2223, 2224, 2225]
CR_list_2020 = [2235, 2236, 2237, 2238, 2239]
omni_valid, model_valid, wsa_valid, eswf_valid = [], [], [], []

for CR_lists in [CR_list_2012, CR_list_2013, CR_list_2014, CR_list_2015, CR_list_2016, CR_list_2017, CR_list_2018, CR_list_2019, CR_list_2020]:
    print("{}".format(CR_lists))
    for CR in tqdm(CR_lists):
        _, omni_speeds = OMNI(CR)
        _, wsa_speeds = WSA_Enlil(CR)
        _, eswf3_2speeds = ESWF3_2(CR)
        _, model_speeds = Model(CR)
        
        omni_speeds = np.array(omni_speeds)
        wsa_speeds = np.array(wsa_speeds)
        eswf3_2speeds = np.array(eswf3_2speeds)
        model_speeds = np.array(model_speeds)
    
        valid_index = ~np.isnan(omni_speeds) & ~np.isnan(model_speeds) & ~np.isnan(wsa_speeds) & ~np.isnan(eswf3_2speeds)

        omni_valid.extend(omni_speeds[valid_index])
        model_valid.extend(model_speeds[valid_index])
        wsa_valid.extend(wsa_speeds[valid_index])
        eswf_valid.extend(eswf3_2speeds[valid_index])
        
    model_pcc, _ = pearsonr(omni_valid, model_valid)
    wsa_pcc, _ = pearsonr(omni_valid, wsa_valid)
    eswf_pcc, _ = pearsonr(omni_valid, eswf_valid)
    
    # PCC 계산
    print("model PCC: {}".format(np.round(model_pcc, 2)))
    print("wsa PCC: {}".format(np.round(wsa_pcc, 2)))
    print("eswf PCC: {}".format(np.round(eswf_pcc, 2)), '\n')
    
    omni_valid, model_valid, wsa_valid, eswf_valid = [], [], [], []